<a href="https://colab.research.google.com/github/anilbhatt1/EVA4P1_S15A_Depth_FG_Detection_DataPrep/blob/master/EVA4P1_S15_OpenCV_FG_Identification_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Fri May 15 10:23:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
import glob
import argparse
import matplotlib
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import skimage
from skimage.transform import resize
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from datetime import datetime 
import gc
sns.set()
from numpy import asarray

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [79]:
log_fg_bg = open('/content/gdrive/My Drive/FG_BG_Filename_Logs.txt',"r")
content_list = log_fg_bg.readlines()
print('No: of records in log_fg_bg:',len(content_list))
print(content_list[0])
log_fg_bg.close()

No: of records in log_fg_bg: 400000
Img_fg_bg_1.jpg,Img21.jpg,Image41.png,(192, 192, 108, 156, 51, 3, 159, 3, 51, 159, 159, 159)



In [68]:
random_lst = []
for i in range(3): 
    x = np.random.randint(low=1, high=400000, size=1)[0]
    random_lst.append(x)
print(random_lst)    

[310962, 17319, 49735]


In [73]:
FG_BG_depth_zip     =  ZipFile("/content/gdrive/My Drive/FG_BG_Depth_400K.zip", 'r')
FG_BG_depth_zip_files = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]
print('len(FG_BG_depth_zip_files)',len(FG_BG_depth_zip_files))
print(FG_BG_depth_zip_files[0])

len(FG_BG_depth_zip_files) 400000
FG_BG_Depth_0_400K/Img_fg_bg_214327_depth.jpg


In [0]:
path_bg_depth = '/content/gdrive/My Drive/BG_BGFlip_Depth/'
path_fg_mask  = '/content/sample_data/fg_mask/'

In [0]:
for i in range(len(random_lst)):
    file_id = random_lst[i]
    fg_bg_depth_name = 'FG_BG_Depth_0_400K/Img_fg_bg_' + str(file_id) + '_depth.jpg'
    imgdata = FG_BG_depth_zip.read(fg_bg_depth_name)
    img     = Image.open(io.BytesIO(imgdata))
    fg_bg_depth_array = np.array(img)

    bg_name = content_list[file_id-1].split(',')[1].split('.')[0]
    if file_id % 2 == 0:
      bg_depth_name = bg_name + '_flip_depth.jpg'
    else:
      bg_depth_name = bg_name + '_depth.jpg'  
    
    img = Image.open(f'{path_bg_depth}{bg_depth_name}')
    bg_depth_array = np.array(img)

    delta_image = fg_bg_depth_array - bg_depth_array
    contour_image = delta_image.copy()
    ret, thresh = cv2.threshold(contour_image, 127, 255, 0)
    contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE) 
    out_mask = np.zeros_like(contour_image)
    fg_mask = cv2.drawContours(out_mask, contours,-1, 255, cv2.FILLED)
    fg_mask_name = 'Img_fg_detect_' + str(file_id) + '.jpg'
    cv2.imwrite(f'{path_fg_mask}{fg_mask_name}',fg_mask)

### Just for verification Purpose we are displaying - No need to run

In [0]:
FG_BG_zip     =  ZipFile("/content/gdrive/My Drive/FG_BG_400K.zip", 'r')
FG_BG_zip_files = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]

In [78]:
FG_BG_zip_files[1]

'FG_BG_400K/Img_fg_bg_147436.jpg'

In [0]:
path_fg_bg    = '/content/sample_data/fg_bg/'

In [0]:
for i in range(len(random_lst)):
    file_id    = random_lst[i]
    fg_bg_name = 'FG_BG_400K/Img_fg_bg_' + str(file_id) + '.jpg'
    imgdata    = FG_BG_zip.read(fg_bg_name)
    img        = Image.open(io.BytesIO(imgdata))
    fg_bg_save_name = 'Img_fg_bg_' + str(file_id) + '.jpg'
    img.save(f'{path_fg_bg}{fg_bg_save_name}') 